In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb

In [2]:
#try overlapping and smaller regions
def extract_feature_vector(img):
    splits=[0,50,100,150,200]
    blocks=[]
    for i in range(4):
        for j in range(4):
            blocks.append(list(img[splits[i]:splits[i+1], splits[j]:splits[j+1]]))
    
   
    histograms=[{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}]
    for i in range(256):
        for j in range(16):
            histograms[j][i]=0
    
    count=0
    for i in blocks:
        for m in i:
            for n in m:
                histograms[count][n]+=1
                
        count+=1
    
    feature_vector=[]
    
    for i in histograms:
        feature_vector=feature_vector+list(i.values())
        
    return feature_vector

## My own coded LBP

In [3]:
directory = os.listdir('../../dataset_lbp/train')
df=pd.read_csv('../recognition/ids.csv', names=['image','class'], header=None)

X_train=[]
Y_train=[]

for file in directory:
    img = cv2.imread("../../dataset_lbp/train/" + file, cv2.IMREAD_GRAYSCALE)
    
    img_resized = cv2.resize(img, (200, 200), interpolation = cv2.INTER_LINEAR)
    
    feature_vector=extract_feature_vector(img_resized)
    
    X_train.append(feature_vector)
    Y_train.append( list(df[df.image=='train/'+file]['class'])[0] )

In [4]:
directory = os.listdir('../../dataset_lbp/test')
df=pd.read_csv('../recognition/ids.csv', names=['image','class'], header=None)

X_test=[]
Y_test=[]

for file in directory:
    img = cv2.imread("../../dataset_lbp/test/" + file, cv2.IMREAD_GRAYSCALE)
    
    img_resized = cv2.resize(img, (200, 200), interpolation = cv2.INTER_LINEAR)
    
    feature_vector=extract_feature_vector(img_resized)
            
    X_test.append(feature_vector)
    Y_test.append( list(df[df.image=='train/'+file]['class'])[0] )

In [5]:
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.004016064257028112


In [6]:
xg_model = xgb.XGBClassifier()
xg_model.fit(X_train, Y_train)
Y_predicted = xg_model.predict(X_test)
#print(classification_report(Y_test, Y_predicted))
print("Accuracy:",metrics.accuracy_score(Y_test, Y_predicted))

C:\Users\marko\anaconda3\envs\ids-clone\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:54:28] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.01606425702811245


## Sklearn LBP
- https://scikit-image.org/docs/stable/api/skimage.feature.html

In [40]:
from skimage.feature import local_binary_pattern, hog

In [51]:
directory = os.listdir('../../dataset_cropped/train_cropped')
#print(directory)

X_train=[]
Y_train=[]
for file in directory:
    img = cv2.imread("../../dataset_cropped/train_cropped/" + file, cv2.IMREAD_COLOR)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
       
    lbp = local_binary_pattern(img_gray, 8, 1, 'uniform')
    
    lbp = cv2.resize(lbp, (200, 200), interpolation = cv2.INTER_LINEAR)
    feature_vector=extract_feature_vector(lbp)
    
    X_train.append(feature_vector)
    Y_train.append( list(df[df.image=='train/'+file]['class'])[0] )

In [52]:
directory = os.listdir('../../dataset_cropped/test_cropped')
#print(directory)

X_test=[]
Y_test=[]
for file in directory:
    img = cv2.imread("../../dataset_cropped/test_cropped/" + file, cv2.IMREAD_COLOR)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   
    lbp = local_binary_pattern(img_gray, 8, 1, 'uniform')
    lbp = cv2.resize(lbp, (200, 200), interpolation = cv2.INTER_LINEAR)
    feature_vector=extract_feature_vector(lbp)
    
    X_test.append(feature_vector)
    Y_test.append( list(df[df.image=='train/'+file]['class'])[0] )

In [53]:
np.array(X_train).shape

(750, 4096)

In [54]:
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.0


In [30]:
xg_model = xgb.XGBClassifier()
xg_model.fit(X_train, Y_train)
Y_predicted = xg_model.predict(X_test)
#print(classification_report(Y_test, Y_predicted))
print("Accuracy:",metrics.accuracy_score(Y_test, Y_predicted))

C:\Users\marko\anaconda3\envs\ids-clone\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:03:30] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.0
